In [102]:
import numpy as np
from queue import PriorityQueue
import copy
import math

In [103]:
# class <Node> represents a node in a graph which possess informations like
# <pos> which is the coordinate of current position of tge node,
# <action> which is action taken by parent to reach the node
# <parent> which is the parent of the node
class Node :
    # constructor for class <Node>
    def __init__(self, M, pos, redPos, action, parent, g, h) :
        self.M = M
        self.pos = pos             # type : <Coordinate> class
        self.redPos = redPos
        self.action = action
        self.parent = parent        # type : <Node> class
        self.g = g
        self.h = h

    def __lt__(self, other) :
        return (self.g+self.h < other.g+other.h)


In [104]:
class Matrix :

    def __init__(self, M) :
        self.M = M

    # function <__eq__> overrides "==" operator so whenever we write 
    # "matrix1 == matrx2", it will check whether all corresponding elements
    # of both matrices are equal
    def __eq__(self, other) :
        n = other.M.shape[0]
        for i in range(n) :
            for j in range(n) :
                if(self.M[i, j] != other.M[i,j]) :
                    return False
        return True

    # overriding hashing function
    def __hash__(self) :
        return hash(str(self.M))


In [105]:
# class <Coordinate> represents the x-coordinate and y-coordinate of a node        
class Coordinate :
    def __init__(self, x, y) :
        self.x = x
        self.y = y
        
    # function <__eq__> overrides "==" operator so whenever we write 
    # "coord1 == coord2", it will check whether x- and y-coordinate of coord1
    # and coord2 are equal, respectively
    def __eq__(self, other) :
        return self.x == other.x and self.y == other.y
    
    # to return coordinate in parenthesis
    def toString(self) :
        return '(' + str(self.x) + ', ' + str(self.y) + ')'

In [106]:
# function <printPath> prints path using recursion from <destination> node
# and going back to the starting node and start printing nodes from starting
# node to destination node
def printPath(destination) :
    if(destination is None) : return 0
    temp = printPath(destination.parent)
    print(destination.action, destination.pos.toString())
    return 1 + temp

In [107]:
#class Environment contains five fields
# M => matrix which represents the graph in matrix-form
# agentPos => current position of agent in the graph
# goalPos => destination position of agent in graph    
class Environment :
    
    def __init__(self, M, pos, goalPos, redPos) :
        self.M = M                # type : m*n Matrix
        self.agentPos = pos       # type : <Coordinate> class
        self.goalPos = goalPos    # type : <Coordinate> class
        self.m = M.shape[0]       # number of rows of M
        self.n = M.shape[1]       # number of columns of M
        self.redPos = redPos
    
    def updateState(self, action, curr) :
        r = curr.x              
        c = curr.y               
        if(action == 'left') :        
            if(c <= 0 or self.M[r][c-1] != 0) : return False        #if action is invalid
            self.M[r][c], self.M[r][c-1] = (
                    self.M[r][c-1], self.M[r][c])
            print(M[r][c],  M[r][c-1])
            self.redPos = Coordinate(r, c-1)   
            self.agentPos = Coordinate(r, c)
            return True
        elif (action == 'right') :     
            if(c >= self.n-1 or self.M[r][c+1] != 0) : return False   #if action is invalid
            self.M[r][c], self.M[r][c+1] = (
                    self.M[r][c+1], self.M[r][c])
            self.redPos = Coordinate(r, c+1)   
            self.agentPos = Coordinate(r, c)
            return True
        elif (action == 'up') :        
            if(r <= 0 or self.M[r-1][c] != 0) : return  False      #if action is invalid
            self.M[r-1][c], self.M[r][c] = (
                    self.M[r][c], self.M[r-1][c])
            self.redPos = Coordinate(r-1, c)  
            self.agentPos = Coordinate(r, c)
            return True
        elif (action == 'down') :         
            if(r >= self.m-1 or self.M[r+1][c] != 0) : return False      #if action is invalid
            self.M[r+1][c], self.M[r][c] = (
                    self.M[r][c], self.M[r+1][c])
            self.redPos = Coordinate(r+1, c)
            self.agentPos = Coordinate(r, c)
            return True

        else : return False
        
    def updateStateZero(self, action, curr) :
        r = curr.x               #row number of empty block
        c = curr.y               #column number of empty block
        if(action == 'left') :         #move empty block left 
            if(c <= 0 or self.M[r][c-1] == 0) : return False        #if action is invalid
            self.M[r][c], self.M[r][c-1] = (
                    self.M[r][c-1], self.M[r][c])
            self.agentPos = Coordinate(r, c-1)   #update position of agent
            if(self.M[r][c] == 2) :
                self.redPos = Coordinate(r, c)
            return True
        elif (action == 'right') :     #move empty block right
            if(c >= self.n-1 or self.M[r][c+1] == 0) : return False   #if action is invalid
            self.M[r][c], self.M[r][c+1] = (
                    self.M[r][c+1], self.M[r][c])
            self.agentPos = Coordinate(r, c+1)    #update position of agent
            if(self.M[r][c] == 2) :
                self.redPos = Coordinate(r, c)
            return True
        elif (action == 'up') :        #move empty block up
            if(r <= 0 or self.M[r-1][c] == 0) : return  False      #if action is invalid
            self.M[r-1][c], self.M[r][c] = (
                    self.M[r][c], self.M[r-1][c])
            self.agentPos = Coordinate(r-1, c)   #update position of agent
            if(self.M[r][c] == 2) :
                self.redPos = Coordinate(r, c)
            return True
        elif (action == 'down') :         #move empty block down
            if(r >= self.m-1 or self.M[r+1][c] == 0) : return False      #if action is invalid
            self.M[r+1][c], self.M[r][c] = (
                    self.M[r][c], self.M[r+1][c])
            self.agentPos = Coordinate(r+1, c)  #update position of agent
            if(self.M[r][c] == 2) :
                self.redPos = Coordinate(r, c)
            return True

        else : return False
        
    def providePerception(self) :
        # <__eq__> function of class <Coordinate> will be called
        return self.redPos == self.goalPos 

    def manhattanDist(self, curr) :
        return math.fabs(self.goalPos.x-curr.x) + math.fabs(self.goalPos.y-curr.y)

    def sumOfSquares(self, curr, goal) :
        return (curr.x - goal.x)*(curr.x-goal.x) + (curr.y - goal.y)*(curr.y-goal.y)

In [108]:
class Agent :
    
    def takeAction(self, envObj, action, curr) :
        return envObj.updateState(action, curr)
    
    def takeActionZero(self, envObj, action, curr) :
        return envObj.updateStateZero(action, curr)

    def getPerception(self, envObj) :
        return envObj.providePerception()

In [117]:
q = PriorityQueue() # initialize priority queue
# <M> is the matrix representing the graph
# 1 represents blocked positions
# 0 represents walkable positions
# M = np.array([[1, 0, 0, 1, 1, 1, 0, 1, 1, 1 ],
#               [1, 0, 0, 0, 1, 1, 1, 0, 1, 1 ],
#               [1, 1, 0, 0, 1, 1, 0, 1, 0, 1 ],
#               [0, 0, 0, 0, 0, 0, 0, 0, 0, 1 ],
#               [1, 1, 1, 0, 0, 1, 1, 0, 1, 0 ],
#               [1, 0, 0, 1, 0, 1, 0, 1, 0, 0 ],
#               [1, 0, 0, 0, 0, 0, 0, 0, 0, 1 ],
#               [1, 0, 1, 0, 1, 0, 0, 1, 1, 1 ],
#               [1, 1, 0, 0, 0, 0, 1, 0, 0, 1 ]])
m, n = map(int, input("Enter the shape of matrix (Eg m n)").split())
red_i, red_j = map(int, input("Enter red position (Eg i j)").split())
empty_i, empty_j = map(int, input("Enter empty position (Eg i j)").split())
# M = np.array([[1, 1, 0, 1], 
#              [1, 1, 1, 1],
#              [1, 2, 1, 1],
#              [1, 1, 1, 1]])
# <start> represents starting position 
M = np.ones((m, n))
M[red_i, red_j] = 2
M[empty_i, empty_j] = 0
print ("__________________________")
print ("GRID")
print (M)
print ("__________________________")
print ("Heuristic = Manhattan Distance")
print ("Moves Allowed = left,right,up,down")
start = Coordinate(empty_i, empty_j)
print ("__________________________")
print ("Source = ", start.toString())
# <end> represents goal position
end = Coordinate(0, M.shape[1]-1)
print ("Goal = ", end.toString())
redPos = Coordinate(red_i, red_j)
print("Red = ", redPos.toString())
# creating an instance <envObj> of class Environment initialized with matrix M,
# starting position <start> and goal position <end>
envObj = Environment(M, start, end, redPos)
agent = Agent()
s = set()
# putting <start> node into queue
# A_Track = M.copy().astype(np.float)
# A_Track[A_Track == 1] = float('Inf')
q.put(Node(M.copy(), start, redPos, 'None', None, 0, envObj.manhattanDist(envObj.redPos)))
s.add(Matrix(M))
# envObj.M[start.x][start.y] = 1
destination = None
# c = 1
# print("TRACKING A* ALGORITHM : ")
while(not q.empty()) :
    curr = q.get()
    envObj.M = curr.M.copy()
    envObj.redPos = copy.copy(curr.redPos)
    envObj.agentPos = copy.copy(curr.redPos)
    if(curr.parent is not None) :
        print("parent : \n", curr.parent.M, "\n action = ", curr.action)
    print("\nredPos : ", curr.redPos.toString())
    print("\nzero : ", curr.pos.toString())
    print(envObj.M)
    if(agent.getPerception(envObj)) : 
        print("Destination Reached")
        destination = curr
        break
    actionList = ['left', 'right', 'up', 'down']
    redPos = curr.redPos
    currPos = curr.pos
#     c+=1
    flagRed = False
    for action in actionList :
        envObj.M = curr.M.copy()
        envObj.redPos = copy.copy(redPos)
        envObj.agentPos = copy.copy(currPos)
        flag = agent.takeAction(envObj, action, copy.copy(redPos))
        m = Matrix(copy.copy(envObj.M))
        if(flag and (m not in s)) :
            flagRed = True
            q.put(Node(envObj.M.copy(), copy.copy(envObj.agentPos), copy.copy(envObj.redPos), action, curr
                      , curr.g+1, envObj.manhattanDist(envObj.redPos)))
            s.add(m)

    for action in actionList :
        envObj.M = curr.M.copy()
        envObj.redPos = copy.copy(redPos)
        envObj.agentPos = copy.copy(currPos)
        flag = agent.takeActionZero(envObj, action, copy.copy(currPos))
        m = Matrix(copy.copy(envObj.M))
        if(flag and (m not in s)) :
            q.put(Node(envObj.M.copy(), copy.copy(envObj.agentPos), copy.copy(envObj.redPos), action, curr
                       , curr.g+1, envObj.manhattanDist(envObj.redPos)))
            s.add(m)

    
if(destination is None) :
    print("Not reachable")
else :
    print ("__________________________")
    print ("SHORTEST PATH:")
    cost = printPath(destination)
    print ("__________________________")
    print ("cost = ",cost - 1)
    print ("__________________________")

Enter the shape of matrix (Eg m n)4 4
Enter red position (Eg i j)2 1
Enter empty position (Eg i j)0 3
__________________________
GRID
[[1. 1. 1. 0.]
 [1. 1. 1. 1.]
 [1. 2. 1. 1.]
 [1. 1. 1. 1.]]
__________________________
Heuristic = Manhattan Distance
Moves Allowed = left,right,up,down
__________________________
Source =  (0, 3)
Goal =  (0, 3)
Red =  (2, 1)

redPos :  (2, 1)

zero :  (0, 3)
[[1. 1. 1. 0.]
 [1. 1. 1. 1.]
 [1. 2. 1. 1.]
 [1. 1. 1. 1.]]
parent : 
 [[1. 1. 1. 0.]
 [1. 1. 1. 1.]
 [1. 2. 1. 1.]
 [1. 1. 1. 1.]] 
 action =  left

redPos :  (2, 1)

zero :  (0, 2)
[[1. 1. 0. 1.]
 [1. 1. 1. 1.]
 [1. 2. 1. 1.]
 [1. 1. 1. 1.]]
parent : 
 [[1. 1. 1. 0.]
 [1. 1. 1. 1.]
 [1. 2. 1. 1.]
 [1. 1. 1. 1.]] 
 action =  down

redPos :  (2, 1)

zero :  (1, 3)
[[1. 1. 1. 1.]
 [1. 1. 1. 0.]
 [1. 2. 1. 1.]
 [1. 1. 1. 1.]]
parent : 
 [[1. 1. 0. 1.]
 [1. 1. 1. 1.]
 [1. 2. 1. 1.]
 [1. 1. 1. 1.]] 
 action =  left

redPos :  (2, 1)

zero :  (0, 1)
[[1. 0. 1. 1.]
 [1. 1. 1. 1.]
 [1. 2. 1. 1.]
 [1. 1. 

 [[1. 0. 1. 1.]
 [2. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]] 
 action =  left

redPos :  (1, 0)

zero :  (0, 0)
[[0. 1. 1. 1.]
 [2. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
parent : 
 [[1. 1. 1. 1.]
 [1. 1. 1. 2.]
 [1. 1. 0. 1.]
 [1. 1. 1. 1.]] 
 action =  right

redPos :  (1, 3)

zero :  (2, 3)
[[1. 1. 1. 1.]
 [1. 1. 1. 2.]
 [1. 1. 1. 0.]
 [1. 1. 1. 1.]]
parent : 
 [[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [2. 1. 1. 1.]
 [1. 0. 1. 1.]] 
 action =  left

redPos :  (2, 0)

zero :  (3, 0)
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [2. 1. 1. 1.]
 [0. 1. 1. 1.]]
parent : 
 [[1. 1. 1. 1.]
 [1. 1. 1. 2.]
 [1. 1. 0. 1.]
 [1. 1. 1. 1.]] 
 action =  left

redPos :  (1, 3)

zero :  (2, 1)
[[1. 1. 1. 1.]
 [1. 1. 1. 2.]
 [1. 0. 1. 1.]
 [1. 1. 1. 1.]]
parent : 
 [[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [0. 1. 1. 1.]
 [1. 2. 1. 1.]] 
 action =  up

redPos :  (3, 1)

zero :  (1, 0)
[[1. 1. 1. 1.]
 [0. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 2. 1. 1.]]
parent : 
 [[0. 1. 1. 1.]
 [2. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]] 
 action =  up

red